# Exploratory Data Analysis with SparkR

[**Introduction to Apache Spark with R by J. A. Dianes**](https://github.com/jadianes/spark-r-notebooks)

## Creating a SparkSQL context and loading data

## Exploring property value

- Box plots for categorical variables (E.g. region)
- Scatter plots or bubble plots for two or three numerical ordinal variables